# Gatsiva Sample Analysis
## Average Return After Varying Correction Levels

This notebook shows how to visualize average returns after varying levels of a price correction event utilizing the Gatsiva API and graphing with Plotly.

API access is **currently limited to beta testers and collaborators**. For more information on how to utilize the Gatsiva API or to request access and an API key, please visit the [Gatsiva Website](https://gatsiva.com). For more information on the Gatsiva API, please visit the [Gatsiva API Documentation](https://gatsiva.com/docs).

## Enter Input Information Here

In [ ]:
import numpy as np

symbol = 'BTC:USD:daily'
test_periods = np.arange(-0.10,0,0.02)
periods = 45
usingdatafrom = "2017-01-09"
usingdatato = "2018-01-17"
apikey = "<your_api_key>"  # enter your own API key here (to request an API key visit gatsiva.com)

#### Python code to get the results - see results further below

In [ ]:
# Install Prerequesities if needed
#
# Run this (uncomment below) to install / upgrade Plotly
#!pip install plotly --upgrade
#!pip install colorlover

In [ ]:
# Import utility libraries we will need
import requests
import json
import numpy as np
import pandas as pd
import warnings
import colorlover as cl

# Import plotly for graphing awesomeness
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

# To allow the jupyter notebook to render in offline mode
init_notebook_mode(connected=True)

# Ignore warnings
warnings.filterwarnings('ignore')

data_run_list = [] # List for mean return information
obs_num_list = [] # List for storing number of observations in each run

# Setup the headers for the request
bearer_token = 'Bearer ' + apikey
my_headers = {'Accept': 'application/json', 'Authorization': bearer_token}

for p in test_periods:

    # Print our progress
    print('Running test period ' + str(p));
    
    # Create the condition
    condition = 'price change percentage(1) < ' + str(p)

    # Form the request
    request_data = {'symbol':symbol,'condition_id':1,'condition':condition,"periods":periods,'from':usingdatafrom,'to':usingdatato}

    # Send the request
    analytic = requests.post('https://api.gatsiva.com/api/v1/conditions/return_profile',json=request_data,headers=my_headers)

    # Pull out the results and format it into a pandas data frame
    results = analytic.json()['results']
    df = pd.read_json(json.dumps(results), orient='records')

    # Add the mean and number of observations to our list
    data_run_list.append(df['mean'])   
    obs_num_list.append(results[0]['n'])

# Set up the color scales
bupu = cl.scales['3']['seq']['BuPu']
colorscale = cl.interp( bupu, 16 )
colorscale.reverse()

# Now create the data structure needed for plotly

i=0
data = []

while i < len(test_periods):
    data.append(
        Scatter(
            x=df['returnperiod'],
            y=data_run_list[i],
            name='> ' + str(test_periods[i]*100) + '% (' + str(obs_num_list[i]) + ' obs)',
            marker = dict(
                color = colorscale[i]
            )
        )
    )
    i = i + 1

In [ ]:
# Create the layout needed for plotly
layout = Layout(
    title='Average Return After Correction for ' + symbol + ' from ' + usingdatafrom + ' to ' + usingdatato,
    yaxis=dict(title='return'),
    xaxis=dict(title='days after correction')
)

# Create the figure
fig = Figure(data=data, layout=layout)

## Results

In [ ]:
# Plot the result
iplot(fig)